In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

In [16]:
driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
url = 'https://gearvn.com/collections/laptop'
driver.get(url)

wait = WebDriverWait(driver, 30)
# S·ª≠ d·ª•ng JavaScript ƒë·ªÉ ·∫©n pop-up
driver.execute_script("document.getElementById('popup-promotion').style.display = 'none';")

def is_view_more_button_displayed():
    try:
        view_more_button = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, 'load_more'))
        )
        return view_more_button.is_displayed()
    except:
        return False

def get_product_count():
    # S·ª≠ d·ª•ng m·ªôt selector ch√≠nh x√°c h∆°n ƒë·ªÉ ƒë·∫øm s·ªë l∆∞·ª£ng s·∫£n ph·∫©m
    product_elements = driver.find_elements(By.CLASS_NAME, 'col-xl-3.col-lg-3.col-6.proloop.ajaxloop.loaded')
    return len(product_elements)
def is_product_count_changed(previous_product_count):
    current_product_count = get_product_count()
    return current_product_count != previous_product_count

# V√≤ng l·∫∑p ƒë·ªÉ click v√†o n√∫t "Xem th√™m s·∫£n ph·∫©m" cho ƒë·∫øn khi n√≥ kh√¥ng c√≤n hi·ªÉn th·ªã
while is_view_more_button_displayed():
    initial_product_count = get_product_count()    
    view_more_button = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.ID, 'load_more'))
    )
    driver.execute_script("arguments[0].scrollIntoView();", view_more_button)
    
    # ƒê·ª£i cho d·ªØ li·ªáu m·ªõi t·∫£i l√™n
    try:
        WebDriverWait(driver, 30).until(lambda driver: is_product_count_changed(initial_product_count))
    except TimeoutException:
        pass
    # Th·ª±c hi·ªán click l·∫°i n·∫øu c√≥ l·ªói ElementClickInterceptedException
    try:
        view_more_button.click()
    except ElementClickInterceptedException:
        pass
        view_more_button.click()

    # ƒê·ª£i m·ªôt kho·∫£ng th·ªùi gian tr∆∞·ªõc khi ki·ªÉm tra s·ªë l∆∞·ª£ng s·∫£n ph·∫©m, n·∫øu kh√¥ng c√≥ s·ª± thay ƒë·ªïi th√¨ break (T·ª©c l√† ƒë√£ hi·ªÉn th·ªã to√†n b·ªô s·∫£n ph·∫©m)
    sleep(3)
    if not is_product_count_changed(initial_product_count):
        break

# ƒê·ª£i cho c√°c li√™n k·∫øt s·∫£n ph·∫©m tr·ªü n√™n hi·ªÉn th·ªã
product_links = wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'proloop-name')))

# Tr√≠ch xu·∫•t thu·ªôc t√≠nh href t·ª´ c√°c li√™n k·∫øt
all_product_links = [link.find_element(By.TAG_NAME, 'a').get_attribute('href') for link in product_links]

print(all_product_links)

driver.quit()

['https://gearvn.com/products/laptop-lenovo-ideapad-slim-3-15abr8-82xm00ehvn', 'https://gearvn.com/products/laptop-gaming-msi-stealth-18-ai-studio-a1vhg-021vn', 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-5-15iru9-83d0000evn', 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-5-15iru9-83d00003vn', 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-3-15abr8-82xm00ejvn', 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-5-14imh9-83da0020vn', 'https://gearvn.com/products/laptop-asus-vivobook-14-oled-a1405va-km257w', 'https://gearvn.com/products/laptop-asus-vivobook-15-oled-a1505va-l1491w', 'https://gearvn.com/products/laptop-gaming-lenovo-loq-15irx9-83dv00d5vn', 'https://gearvn.com/products/laptop-lenovo-thinkpad-e16-gen-1-21jn005mvn', 'https://gearvn.com/products/laptop-gaming-lenovo-loq-15irx9-83dv00ervn', 'https://gearvn.com/products/laptop-gaming-asus-rog-zephyrus-g16-gu605mv-qr196ws', 'https://gearvn.com/products/laptop-gaming-asus-tuf-gaming-a15-fa507uv-

In [17]:
print(len(all_product_links))

277


In [268]:
def get_product_info(all_product_links):
    driver.get(all_product_links)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    #PRODUCT_NAME
    try:
        product_name = soup.find('div', class_ = 'product-name').text.strip()
    except:
        product_name = ''
        #PRODUCT_URL
    try:
        product_url = soup.find('link', {'rel': 'canonical'}).get('href')
    except:
        product_url = ''
    try:
        discounted_price = soup.find('span', class_ = 'pro-price a').text.strip()
    except:
        discounted_price = ''
    try:
        original_price = soup.find('del').text.strip()
    except:
        original_price = discounted_price    
    try:   
        discount_percentage = soup.find('span', class_ = 'pro-percent').text.strip().replace('-','')
    except:
        discount_percentage = ''
    
    # T·∫°o m·ªôt t·ª´ ƒëi·ªÉn ƒë·ªÉ l∆∞u c√°c gi√° tr·ªã
    promotions = []

    try:
        gift_promo_elements = soup.find_all('li', class_='higtlight')
        for gift_promo_element in gift_promo_elements:
        # Tr√≠ch xu·∫•t th√¥ng tin t·ª´ m·ªói ph·∫ßn t·ª≠ v√† th√™m v√†o t·ª´ ƒëi·ªÉn
            promotion_text = gift_promo_element.text.strip()
            promotions.append(promotion_text )
    except AttributeError:
        pass

    try:
        promotion_section = soup.find('div', class_='product-desc-short prtab2')
        # Ki·ªÉm tra xem ph·∫ßn t·ª≠ ch·ª©a th√¥ng tin qu√† t·∫∑ng c√≥ t·ªìn t·∫°i kh√¥ng
        if promotion_section.find('strong', string='Qu√† t·∫∑ng:'):
            gift_section = promotion_section.find('strong', string='Qu√† t·∫∑ng:')
            if gift_section.find_next('span').find_all('br'):
                gift_section_2 = gift_section.find_next('span').find_all('br')
                for br_tag in gift_section_2:
                    if br_tag.previous_sibling:
                        promotions.append(br_tag.previous_sibling.strip().replace('\xa0',' ').replace('üéÅ ','').replace('üéÅ',''))
                    if br_tag.find_next_sibling:
                        promotions.append(br_tag.next_sibling.strip().replace('\xa0',' ').replace('üéÅ ','').replace('üéÅ',''))
            else:
                gift_find = gift_section.find_parent('p')
                siblings = gift_find.find_next_sibling()

                for sibling in siblings:
                    if sibling.find_all('br'):
                        for br_tag in sibling.find_all('br'):
                            if br_tag.previous_sibling:
                                promotions.append(br_tag.previous_sibling.strip().replace('\xa0',' ').replace('üéÅ ','').replace('üéÅ',''))
                            if br_tag.next_sibling:
                                promotions.append(br_tag.next_sibling.strip().replace('\xa0',' ').replace('üéÅ ','').replace('üéÅ',''))
                    else:
                        promotion_more = sibling.text.strip().replace('\xa0',' ').replace('üéÅ ','').replace('üéÅ','')
                        if promotion_more == '':
                            break
                        else:
                            promotions.append(promotion_more)
        
        if promotion_section.find_next('p'):
            if promotion_section.find_next('p').find_all('br'):
                gift_box = promotion_section.find_next('p').find_all('br')
                for gift in gift_box:
                    gift_content = gift.next_sibling.text.strip().replace('üéÅ ','').replace('üéÅ','').replace('\xa0',' ')
                    promotions.append(gift_content)
            elif promotion_section.find_next('p'):
                if 'üéÅ' in promotion_section.find_next('p').text.strip():
                    gift_text = promotion_section.find_next('p').text.strip().replace('\xa0',' ').replace('üéÅ ','').replace('üéÅ','')
                    promotions.append(gift_text)
            else:
                pass
    except AttributeError:
        pass
    max_promotion = 3
    product_price = {
            'product_name': product_name,
            'discounted_price': discounted_price,
            'original_price': original_price,
            'discount_percentage': discount_percentage,
            'shop_brand': 'GearVN'
        }
    # Th√™m c√°c bi·∫øn khuy·∫øn m·∫°i v√†o dictionary
    for i in range(1, max_promotion + 1):
        key = f'promotion_{i}'
        # N·∫øu i v∆∞·ª£t qu√° s·ªë l∆∞·ª£ng th·ª±c s·ª± c·ªßa khuy·∫øn m·∫°i, thi·∫øt l·∫≠p gi√° tr·ªã l√† None
        if i <= len(promotions):
            value = promotions[i-1]
        else:
            value = ''

        product_price[key] = value

    product_price['product_url'] = product_url

    return product_price

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
# Duy·ªát qua c√°c ƒë∆∞·ªùng link s·∫£n ph·∫©m v√† l·∫•y th√¥ng tin
data = []
for product_link in all_product_links:
    product_price = get_product_info(product_link)
    data.append(product_price)
    print(product_price)

driver.quit()

{'product_name': 'Laptop Lenovo Ideapad Slim 3 15ABR8 82XM00EHVN', 'discounted_price': '14.990.000‚Ç´', 'original_price': '15.990.000‚Ç´', 'discount_percentage': '6%', 'shop_brand': 'GearVN', 'promotion_1': 'Balo Lenovo Office', 'promotion_2': '', 'promotion_3': '', 'product_url': 'https://gearvn.com/products/laptop-lenovo-ideapad-slim-3-15abr8-82xm00ehvn'}
{'product_name': 'Laptop gaming MSI Stealth 18 AI Studio A1VHG 021VN', 'discounted_price': '93.990.000‚Ç´', 'original_price': '94.990.000‚Ç´', 'discount_percentage': '1%', 'shop_brand': 'GearVN', 'promotion_1': 'Stealth Agent Backpack ( k√®m m√°y)', 'promotion_2': 'Chu·ªôt MSI M99 Pro ( k√®m m√°y)', 'promotion_3': '', 'product_url': 'https://gearvn.com/products/laptop-gaming-msi-stealth-18-ai-studio-a1vhg-021vn'}
{'product_name': 'Laptop Lenovo Ideapad Slim 5 15IRU9 83D0000EVN', 'discounted_price': '19.490.000‚Ç´', 'original_price': '20.990.000‚Ç´', 'discount_percentage': '7%', 'shop_brand': 'GearVN', 'promotion_1': 'Balo Lenovo Off

In [269]:
import pandas as pd
df = pd.DataFrame(data)
df

product_name discounted_price  \
0       Laptop Lenovo Ideapad Slim 3 15ABR8 82XM00EHVN      14.990.000‚Ç´   
1    Laptop gaming MSI Stealth 18 AI Studio A1VHG 0...      93.990.000‚Ç´   
2       Laptop Lenovo Ideapad Slim 5 15IRU9 83D0000EVN      19.490.000‚Ç´   
3       Laptop Lenovo Ideapad Slim 5 15IRU9 83D00003VN      20.490.000‚Ç´   
4       Laptop Lenovo Ideapad Slim 3 15ABR8 82XM00EJVN      13.490.000‚Ç´   
..                                                 ...              ...   
272                  Laptop Acer Aspire 7 A715 76 57CY      14.490.000‚Ç´   
273  Laptop gaming ASUS ROG Strix SCAR 16 G634JZR N...     101.990.000‚Ç´   
274                 Laptop Acer Swift X SFX14 71G 75CV      39.990.000‚Ç´   
275  Laptop gaming Lenovo Yoga Pro 7 14IRH8 82Y70050VN      36.990.000‚Ç´   
276     Laptop gaming ASUS ROG Flow Z13 GZ301VU MU301W      52.990.000‚Ç´   

    original_price discount_percentage shop_brand  \
0      15.990.000‚Ç´                  6%     GearVN   
1      94.990.000‚Ç´                  1%     GearVN   
2      20.990.000‚Ç´                  7%     GearVN   
3      21.990.000‚Ç´                  7%     GearVN   
4      13.990.000‚Ç´                  4%     GearVN   
..             ...                 ...        ...   
272    16.490.000‚Ç´                 12%     GearVN   
273   110.000.000‚Ç´                  7%     GearVN   
274    40.990.000‚Ç´                  2%     GearVN   
275    94.990.000‚Ç´                         GearVN   
276    57.990.000‚Ç´                  9%     GearVN   

                                      promotion_1  \
0                              Balo Lenovo Office   
1               Stealth Agent Backpack ( k√®m m√°y)   
2                              Balo Lenovo Office   
3                              Balo Lenovo Office   
4                              Balo Lenovo Office   
..                                            ...   
272                                                 
273              Balo ASUS ROG Backpack (k√®m m√°y)   
274  Gi·∫£m th√™m 500.000ƒë cho HSSV (ƒë·∫øn 30.09.2023)   
275                                                 
276                 T√∫i ROG Flow Sleeve (K√®m m√°y)   

                      promotion_2 promotion_3  \
0                                               
1    Chu·ªôt MSI M99 Pro ( k√®m m√°y)               
2                                               
3                                               
4                                               
..                            ...         ...   
272                                             
273                                             
274                                             
275                                             
276                                             

                                           product_url  
0    https://gearvn.com/products/laptop-lenovo-idea...  
1    https://gearvn.com/products/laptop-gaming-msi-...  
2    https://gearvn.com/products/laptop-lenovo-idea...  
3    https://gearvn.com/products/laptop-lenovo-idea...  
4    https://gearvn.com/products/laptop-lenovo-idea...  
..                                                 ...  
272  https://gearvn.com/products/laptop-acer-aspire...  
273  https://gearvn.com/products/laptop-gaming-asus...  
274  https://gearvn.com/products/laptop-acer-swift-...  
275  https://gearvn.com/products/laptop-gaming-leno...  
276  https://gearvn.com/products/laptop-gaming-asus...  

[277 rows x 9 columns]

In [271]:
df.to_excel('gearvn_price.xlsx', index=False)